### Feature Engineering
- Text: TF‑IDF pe titlu.
- Numeric:
- number_of_views → log-transform.
- merchant_rating → scalare.
- listing_date → extragem year, month.


In [35]:
import numpy as np
import pandas as pd

df = pd.read_csv("../data/products_modified.csv")

df["views_log"] = np.log1p(df["number_of_views"])
df["year"] = pd.to_datetime(df["listing_date"]).dt.year
df["month"] = pd.to_datetime(df["listing_date"]).dt.month


### Building the pipeline

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

text_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=100000)

preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_vectorizer, "product_title"),
        ("num", StandardScaler(), ["views_log", "merchant_rating", "year", "month"])
    ]
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(max_iter=2000, n_jobs=4, multi_class="multinomial", class_weight="balanced"))
])

### Training and assessment 

- Metrici: accuracy, precision, recall.
- Vizualizare: matrice de confuzie.


In [44]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

print(df.isna().sum())
df = df.dropna(subset=["product_title"])

# Features și label
X = df[["product_title", "views_log", "merchant_rating", "year", "month"]]
y = df["category_label"]

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Models to test
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, class_weight="balanced", multi_class="multinomial"),
    "Naive Bayes": MultinomialNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Linear SVM": LinearSVC()
}

# Loop through models
for name, model in models.items():
    print(f"\n {name}")
    
    if name == "Naive Bayes":
        pipeline = Pipeline([
            ("preprocessing", ColumnTransformer([
                ("text", TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=100000), "product_title")
            ])),
            ("classifier", model)
        ])
    else:
        pipeline = Pipeline([
            ("preprocessing", ColumnTransformer([
                ("text", TfidfVectorizer(ngram_range=(1,2), min_df=2, max_features=100000), "product_title"),
                ("num", StandardScaler(), ["views_log", "merchant_rating", "year", "month"])
            ])),
            ("classifier", model)
        ])
    
    # Training and assessment
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    
    print("Accuracy:", accuracy_score(y_val, y_pred))
    print(classification_report(y_val, y_pred, zero_division=0))

Unnamed: 0         0
product_id         0
product_title      0
merchant_id        0
category_label     0
product_code       0
number_of_views    0
merchant_rating    0
listing_date       0
views_log          0
year               0
month              0
dtype: int64

🔍 Logistic Regression


c:\Users\uvirg\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9247698504027618
                  precision    recall  f1-score   support

             CPU       0.00      0.00      0.00        16
            CPUs       0.98      0.94      0.96       742
 Digital Cameras       1.00      0.99      0.99       532
     Dishwashers       0.91      0.96      0.93       675
        Freezers       0.87      0.95      0.91       436
 Fridge Freezers       0.96      0.89      0.93      1085
         Fridges       0.85      0.80      0.83       681
      Microwaves       0.98      0.97      0.97       461
    Mobile Phone       0.03      0.09      0.04        11
   Mobile Phones       0.93      0.95      0.94       794
             TVs       0.99      0.97      0.98       701
Washing Machines       0.98      0.94      0.96       794
          fridge       0.04      0.12      0.06        24

        accuracy                           0.92      6952
       macro avg       0.73      0.74      0.73      6952
    weighted avg       0.94      0.92    

c:\Users\uvirg\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.9567031070195627
                  precision    recall  f1-score   support

             CPU       0.00      0.00      0.00        16
            CPUs       0.98      1.00      0.99       742
 Digital Cameras       1.00      0.99      1.00       532
     Dishwashers       0.93      0.96      0.94       675
        Freezers       0.97      0.95      0.96       436
 Fridge Freezers       0.93      0.95      0.94      1085
         Fridges       0.91      0.90      0.91       681
      Microwaves       0.98      0.97      0.97       461
    Mobile Phone       0.00      0.00      0.00        11
   Mobile Phones       0.96      0.99      0.97       794
             TVs       0.99      0.98      0.99       701
Washing Machines       0.97      0.95      0.96       794
          fridge       0.00      0.00      0.00        24

        accuracy                           0.96      6952
       macro avg       0.74      0.74      0.74      6952
    weighted avg       0.95      0.96    

### Save model in file category_model.pkl

In [42]:
import joblib
joblib.dump(pipe, "../models/category_model.pkl")

['../models/category_model.pkl']